In [2]:
import nomic
import json
#api_key_path = "/home/ubuntu/api_keys.json"

#with open(api_key_path, 'r') as j:
#    key = json.loads(j.read())['nomic']

#nomic.login(key)

#- Scientific programming
import numpy as np
import pandas as pd
from tqdm import tqdm

#- Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#- Data
from nomic import AtlasDataset

#- Processing
from fuzzywuzzy import fuzz
from unidecode import unidecode
import re
import datetime

dataset = AtlasDataset('hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-06-10')

embds = dataset.maps[0].embeddings.latent
df = dataset.maps[0].data.df

tags_df = dataset.maps[0].tags.df

df = df.merge(tags_df, on='tweetId')
df.head()

/Users/tianyichen/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2024-06-11 18:08:55.019 | INFO     | nomic.dataset:__init__:829 - Loading existing dataset `hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-06-10`.
100%|██████████| 125/125 [00:02<00:00, 48.62it/s]


,name,party,twitter_lower,years,chamber,state,postedAt,source,tweetId,<115><2017><senate><179>,<118><2024><house><142>
0,Josh Gottheimer D-NJ,D,repjoshg,2021_2022,House,NJ,2022-05-29 13:22:24.171,PolitWoops,1530902377196314624,False,False
1,Ed Markey D-MA,D,senmarkey,2011_2012,Senate,MA,2011-05-13 21:06:26.255,PolitWoops,69146505528868864,False,False
2,Debbie Stabenow D-MI,D,senstabenow,2021_2022,Senate,MI,2022-03-02 01:14:27.293,PolitWoops,1498829050919411712,False,False
3,Ted Cruz R-TX,R,sentedcruz,2019_2020,Senate,TX,2019-10-31 16:48:33.055,PolitWoops,1189947274526384128,False,False
4,Dan Bishop R-NC,R,repdanbishop,2021_2022,House,NC,2021-08-25 00:15:49.917,PolitWoops,1430322991376834560,False,False


In [17]:
data_path = '/Users/tianyichen/Desktop/Research /PhDresearch/Twitters/congressional-twin/data/congress_votes/congress_votes_115-2017_s179.csv'
file_name = data_path.split('congress_votes_')[-1].split('.csv')[0]

temp_ = pd.read_csv(data_path)
vote_info_str = list(temp_.columns)[0]

pattern = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}'
datetime_regex = re.compile(pattern)
match = datetime_regex.search(vote_info_str)

vote_datetime_obj = datetime.datetime.strptime(match.group(), '%Y-%m-%dT%H:%M:%S')

In [18]:
tag_name = '<115><2017><senate><179>'

congress_name_list = np.unique(list(df['name']))
congress_name_list = list(congress_name_list)

tweets_per_congress_person = {
    'global': np.zeros(len(congress_name_list)),
    'issue-specific': np.zeros(len(congress_name_list))
}

average_embds = {
    'global': np.zeros((len(congress_name_list), embds.shape[1])),
    'issue-specific': np.zeros((len(congress_name_list), embds.shape[1]))
}

congress_handles = [list(df['twitter_lower'][df['name'] == cn])[0] for cn in congress_name_list]
party_by_congress_person = [-1 for cp in congress_name_list]

for i, embd in enumerate(tqdm(embds)):
    congress_name_index = congress_name_list.index(df['name'][i])
    time_posted = df['postedAt'][i].to_pydatetime()

    if vote_datetime_obj < time_posted:
        continue

    tweets_per_congress_person['global'][congress_name_index] += 1
    average_embds['global'][congress_name_index] += embd

    if df[tag_name][i]:
        tweets_per_congress_person['issue-specific'][congress_name_index] += 1
        average_embds['issue-specific'][congress_name_index] += embd

    if party_by_congress_person[congress_name_index] == -1:
        party_by_congress_person[congress_name_index] = df['party'][i].lower()

average_embds = {k: np.divide(average_embds[k],  tweets_per_congress_person[k].reshape(-1, 1)) for k in average_embds}
unique_party_labels, party_counts = np.unique(party_by_congress_person, return_counts=True)

100%|██████████| 2904271/2904271 [01:34<00:00, 30652.68it/s]
/var/folders/rl/pr6f4j2d1_5913_lglcbwszc0000gn/T/ipykernel_4961/4091324195.py:36: RuntimeWarning: invalid value encountered in true_divide
  average_embds = {k: np.divide(average_embds[k],  tweets_per_congress_person[k].reshape(-1, 1)) for k in average_embds}


In [19]:
# store everything into a dataframe with each row for a congress person, it has redundant columns. 
data = {'twitter_lower': congress_handles, 'congress_name_list': congress_name_list}

df_congress = pd.DataFrame(data)
df_congress['average_embds_global'] = average_embds['global'].tolist()
df_congress['average_embds_issue_specific'] = average_embds['issue-specific'].tolist()

df_congress = df_congress.merge(df.drop_duplicates(subset='twitter_lower'), on='twitter_lower', how='left')
df_congress.head()

,twitter_lower,congress_name_list,average_embds_global,average_embds_issue_specific,name,party,years,chamber,state,postedAt,source,tweetId,<115><2017><senate><179>,<118><2024><house><142>
0,repfinkenauer,Abby Finkenauer D-IA,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",Abby Finkenauer D-IA,D,2019_2020,House,IA,2020-06-18 15:00:50.141,PolitWoops,1273631764053987328,False,False
1,repspanberger,Abigail Spanberger D-VA,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",Abigail Spanberger D-VA,D,2021_2022,House,VA,2022-04-16 14:37:39.608,PolitWoops,1515338638091030528,False,False
2,repkinzinger,Adam Kinzinger R-IL,"[0.0025755441793512543, 0.03735162318019932, -...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",Adam Kinzinger R-IL,R,2017_2018,House,IL,2018-11-15 16:25:46.373,PolitWoops,1063105789286666240,False,False
3,repadamschiff,Adam Schiff D-CA,"[0.013665779320078302, 0.04313013124946939, -0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",Adam Schiff D-CA,D,2019_2020,House,CA,2019-11-19 20:26:15.846,PolitWoops,1196887433289359360,False,False
4,repadamsmith,Adam Smith D-WA,"[0.007752102992591068, 0.039345022886109776, -...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",Adam Smith D-WA,D,2017_2018,House,WA,2018-08-01 20:44:57.706,PolitWoops,1024757902551261184,False,False


In [21]:
file_name = data_path.split('congress_votes_')[-1].split('.csv')[0]
 
df_vote_result = pd.read_csv(data_path, header=1)

df_vote_result.head()

,person,state,district,vote,name,party
0,300002,TN,NaN,Yea,"Sen. Lamar Alexander [R, 2003-2020]",Republican
1,300018,WA,NaN,Nay,"Sen. Maria Cantwell [D, 2001-2024]",Democrat
2,300019,DE,NaN,Nay,"Sen. Thomas Carper [D, 2001-2024]",Democrat
3,300023,MS,NaN,Yea,"Sen. Thad Cochran [R, 1979-2018]",Republican
4,300025,ME,NaN,Nay,"Sen. Susan Collins [R, 1997-2026]",Republican


In [22]:
df_vote_result['matched-name'] = ''
df_vote_result['twitter-handle'] = ''
df_vote_result['average-embd-global'] = ''
df_vote_result['average-embd-issue-specific'] = ''
df_vote_result['matched-state'] = ''

for i, row in df_vote_result.iterrows():
    name = unidecode(row['name'].split(' [')[0].split('Sen. ')[1]) ## change this to sen or rep based to house or sentate
    party_val = row['party'][0]
    state_val = row['state']
    candidate_names = df_congress.loc[(df_congress['party'] == party_val) & (df_congress['state'] == state_val), 'congress_name_list'].apply(unidecode)
    name_parts = name.split(" ")

    for napa in name_parts:
        if candidate_names.str.contains(napa).any():
            index = candidate_names[candidate_names.str.contains(napa)].index[0]


            df_vote_result.at[i, 'matched-name'] = df_congress.loc[index]['congress_name_list']
            df_vote_result.at[i, 'average-embd-global'] = df_congress.loc[index]['average_embds_global']
            df_vote_result.at[i, 'twitter-handle'] = df_congress.loc[index]['twitter_lower']
            df_vote_result.at[i, 'matched-party']=df_congress.loc[index]['party']
            df_vote_result.at[i, 'matched-state']=df_congress.loc[index]['state']

            if np.isnan(average_embds['issue-specific'][index][0]):
                df_vote_result.at[i, 'average-embd-issue-specific'] = np.nan
            else:
                df_vote_result.at[i, 'average-embd-issue-specific'] = df_congress.loc[index]['average_embds_issue_specific']
        else:
            df_vote_result.at[i, 'matched-name'] = np.nan
            #df_vote_result.at[i, 'average-embd'] = np.nan
            df_vote_result.at[i, 'twitter-handle'] = np.nan
            df_vote_result.at[i, 'matched-party']=np.nan
            df_vote_result.at[i, 'matched-state']=np.nan
            df_vote_result.at[i, 'average-embd-issue-specific'] = np.nan

df_vote_result.head()

,person,state,district,vote,name,party,matched-name,twitter-handle,average-embd-global,average-embd-issue-specific,matched-state,matched-party
0,300002,TN,NaN,Yea,"Sen. Lamar Alexander [R, 2003-2020]",Republican,Lamar Alexander R-TN,senalexander,"[0.009264346055505097, 0.04242450703633876, -0...","[0.009686428719124366, 0.0467718743829986, -0....",TN,R
1,300018,WA,NaN,Nay,"Sen. Maria Cantwell [D, 2001-2024]",Democrat,"Cantwell, Maria",senatorcantwell,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,WA,D
2,300019,DE,NaN,Nay,"Sen. Thomas Carper [D, 2001-2024]",Democrat,"Carper, Thomas",senatorcarper,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,DE,D
3,300023,MS,NaN,Yea,"Sen. Thad Cochran [R, 1979-2018]",Republican,Thad Cochran R-MS,senthadcochran,"[0.01546081527114829, 0.030818887950514285, -0...","[0.017809977192850508, 0.050288344276021926, -...",MS,R
4,300025,ME,NaN,Nay,"Sen. Susan Collins [R, 1997-2026]",Republican,"Collins, Susan",senatorcollins,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,ME,R


In [23]:
df_vote_result.to_csv(f'/Users/tianyichen/Desktop/Research /PhDresearch/Twitters/congressional-twin/data/data/{file_name}.csv')